<a href="https://colab.research.google.com/github/jimregan/wolnelektury-audio-corpus/blob/new-vocab/MLM_scoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/awslabs/mlm-scoring

  Cloning https://github.com/awslabs/mlm-scoring to /tmp/pip-req-build-3jops9b5
  Running command git clone -q https://github.com/awslabs/mlm-scoring /tmp/pip-req-build-3jops9b5
     |████████████████████████████████| 245kB 6.7MB/s 
     |████████████████████████████████| 51kB 8.3MB/s 
     |████████████████████████████████| 645kB 21.3MB/s 
     |████████████████████████████████| 890kB 38.8MB/s 
     |████████████████████████████████| 1.1MB 49.9MB/s 
     |████████████████████████████████| 3.8MB 52.7MB/s 
  Created wheel for mlm: filename=mlm-0.1-cp36-none-any.whl size=37511 sha256=aae4276a1cd1fde8b74506357b3f4e7cc2be9cecd62f4f228c843adb1d9aeed5
  Stored in directory: /tmp/pip-ephem-wheel-cache-79gq_8es/wheels/a9/f1/d7/ae4b7c48fe69aef69b4170ad13c5109cb499daabf027e12728
  Created wheel for gluonnlp: filename=gluonnlp-0.8.3-cp36-none-any.whl size=293540 sha256=233e010cdb9d696dd6da628ec0ee9fac0d2434d5f37f4c21b2a0b758a62ea8f8
  Stored in directory: /root/.cache/pip/wheels/50/6e/32/521aa84d

In [2]:
from transformers import *
model = BertForMaskedLM.from_pretrained("dkleczek/bert-base-polish-uncased-v1")
tokenizer = BertTokenizer.from_pretrained("dkleczek/bert-base-polish-uncased-v1")
nlp = pipeline('fill-mask', model=model, tokenizer=tokenizer)
vocab = None

In [3]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


In [4]:
!pip install mxnet-cu101mkl

     |████████████████████████████████| 712.3MB 25kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


In [5]:
from mlm.scorers import MLMScorer, MLMScorerPT
from mlm.models import get_pretrained
import mxnet as mx
ctxs = [mx.gpu()]
scorer = MLMScorerPT(model, vocab, tokenizer, ctxs)

/usr/local/lib/python3.6/dist-packages/mxnet/optimizer/optimizer.py:167: UserWarning: WARNING: New optimizer gluonnlp.optimizer.lamb.LAMB is overriding existing optimizer mxnet.optimizer.optimizer.LAMB
  Optimizer.opt_registry[name].__name__))


In [6]:
!git clone https://github.com/awslabs/mlm-scoring

Cloning into 'mlm-scoring'...
remote: Enumerating objects: 99, done.
remote: Counting objects: 100% (99/99), done.
remote: Compressing objects: 100% (67/67), done.
remote: Total 99 (delta 31), reused 87 (delta 22), pack-reused 0
Unpacking objects: 100% (99/99), done.


In [ ]:
!head -n 440 mlm-scoring/examples/asr-librispeech-espnet/data/test-clean.am.json

{
  "1089-134686-0000": {
    "hyp_1": {
      "score": -5.1285810470581055,
      "text": " he hoped there would be stew for dinner turnips and carrots and bruised potatoes and fat mutton pieces to be ladled out in thick peppered flower fatten sauce"
    },
    "hyp_10": {
      "score": -8.58308219909668,
      "text": " he hoped there would be stew for dinner turnips and carrots and bruised potatoes and fat mutton pieces to be ladled out in thick peppered flour fattened sauce"
    },
    "hyp_100": {
      "score": -11.993461608886719,
      "text": " he hoped there would be stew for dinner turnips and carrots and bruised potatoes and fat mutton pieces to be latedled out in thick peppered flower fatten sauce"
    },
    "hyp_11": {
      "score": -9.036372184753418,
      "text": " he hoped there would be stew for dinner turnips and carrots and bruised potatoes and fat mutton pieces to be ladled out in the thick peppered flower fatten sauce"
    },
    "hyp_12": {
      "score": -9.

In [7]:
!wget http://poleval.pl/task1/Poleval2020Task1Eval.tar.xz

--2020-08-20 07:40:32--  http://poleval.pl/task1/Poleval2020Task1Eval.tar.xz
Resolving poleval.pl (poleval.pl)... 213.135.36.94
Connecting to poleval.pl (poleval.pl)|213.135.36.94|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12592244 (12M) [application/x-tar]
Saving to: ‘Poleval2020Task1Eval.tar.xz’

Poleval2020Task1Eva 100%[===================>]  12.01M  2.03MB/s    in 6.9s    

2020-08-20 07:40:41 (1.74 MB/s) - ‘Poleval2020Task1Eval.tar.xz’ saved [12592244/12592244]



In [8]:
!tar xf Poleval2020Task1Eval.tar.xz

In [ ]:
import gc
import torch
out = open("/content/scored", "a")
with open(file="/content/Poleval2020Task1Eval/nbest.txt", mode="r", encoding="utf-8") as nbest:
  for line in nbest:
    sentence = []
    words = []
    score = []
    words = line.strip().split(' ')
    id = words[0]
    sentence = ' '.join(words[1:])
    #scorer = MLMScorerPT(model, vocab, tokenizer, ctxs)
    score = scorer.score_sentences([sentence])
    #print("%s %s %f\n" % (id, sentence, score[0]))
    out.write("%s %f\n" % (id, score[0]))
    #del scorer
    torch.cuda.empty_cache()
    gc.collect()


In [ ]:
torch.cuda.empty_cache()